# A geotherm with radiogenic heating and partial melt

In [ ]:
import underworld as uw
import numpy as np
import glucifer
from underworld import function as fn
import os
from mpi4py import MPI

In [ ]:
## All numbers in SI units ##

# model dimensions [km]
length = 270e3
height = 75e3


# desired resolution
lRes = 5e3
hRes = 1e3

In [ ]:
(resI, resJ) = (int(length/lRes),int(height/hRes) )
print resI, resJ

In [ ]:
mymesh = uw.mesh.FeMesh_Cartesian(elementType='Q1/dQ0',
                                elementRes = (resI,resJ),
                                minCoord   = (0.0,-1*height),
                                maxCoord   = (length, 15e3))

In [ ]:
velocityField = uw.mesh.MeshVariable(mymesh,         nodeDofCount=2)
pressureField = uw.mesh.MeshVariable(mymesh.subMesh, nodeDofCount=1)
temperatureField = uw.mesh.MeshVariable(mymesh,      nodeDofCount=1)

meltField = uw.mesh.MeshVariable(mymesh, nodeDofCount=1)

# initialise to be sure
velocityField.data[:] = [0.0,0.0]
temperatureField.data[:] = 0.0

In [ ]:
# create the swarm
myswarm           = uw.swarm.Swarm( mesh=mymesh, particleEscape=True )
swarmLayout       = uw.swarm.layouts.PerCellSpaceFillerLayout( swarm=myswarm, particlesPerCell=20 )

# create swarm variables - tre cool!
materialVariable  = myswarm.add_variable( dataType="int"   , count=1 )
old_melt          = myswarm.add_variable( dataType="double", count=1 )

# populate swarm - should be after add_variables()
myswarm.populate_using_layout( layout=swarmLayout )

In [ ]:
worldParams = {
    'gravity'      : 9.8,
    'cp'           : 1e3, 
    'diffusivity'  : 9.1e-7, # [m^2 s^-1]
    'spreadingV'   : 3.0441400304414e-10, # m*s-1 ~ 0.96 cm/year
    'thermalExp'   : 2.8e-5  # [K^-1]
}

meltParams = {
    'lhf'            : 200, # kJ/kg the latent heat of fusion
    'viscosityChange': 1e3, 
    'densityChange'  : 0.13 
}

airParams = {
    'index'      : 0,
    'density'    : 0., # kg m^-3
    'depth'      : 5.0e3,  # km
    'radiogenic' : 0.0, # W m^-3
}

stickyAirParams = {
    'index'      : 1,
    'density'    : 1.2, # kg m^-3
    'depth'      : 0.0e3,  # km
    'radiogenic' : 0.0, # W m^-3
}
 
crustParams = {
    'index'      : 2,
    'density'    : 2720.0, # kg m^-3
    'depth'      : -60e3,  # km
    'radiogenic' : 7.67e-7 # W m^-3 
}

mantleParams = {
    'index'      : 3,
    'density'    : 3370.0,  # kg m^-3
    'depth'      : -75e3,   # km
    'radiogenic' : 0.0e-6, # W m^-3
}


worldParams['basalheatFlow'] = -0.022/( mantleParams['density'] * worldParams['cp']) # W m^-3/(rho*cp)

In [ ]:
# define material geometry using python loop - slow but explicit
for index in range( len(myswarm.particleCoordinates.data) ):

    coord = myswarm.particleCoordinates.data[index][:]
    
    if   coord[1] > airParams['depth']:
        materialVariable.data[index] = airParams['index']
        
    elif   coord[1] > stickyAirParams['depth']:
        materialVariable.data[index] = stickyAirParams['index']
        
    elif coord[1] > crustParams['depth']:
        materialVariable.data[index] = crustParams['index']
        
    elif coord[1] > mantleParams['depth']:
        materialVariable.data[index] = mantleParams['index']

In [ ]:
matfig = glucifer.Figure()
matfig.append( glucifer.objects.Points(myswarm, materialVariable) )
matfig.show()

In [ ]:
# define special wall boundary index sets
iWalls = mymesh.specialSets['MinI_VertexSet'] + mymesh.specialSets['MaxI_VertexSet']
jWalls = mymesh.specialSets['MinJ_VertexSet'] + mymesh.specialSets['MaxJ_VertexSet']
topWall = mymesh.specialSets['MaxJ_VertexSet']
bottomWall = mymesh.specialSets['MinJ_VertexSet']

# go through local nodes and find switch should be considered air
airNodes = []
for n_i in range(mymesh.nodesLocal):
    ycoord = mymesh.data[n_i][1]
    if ycoord > 0.0:
        airNodes.append(n_i)

airSet = uw.mesh.FeMesh_IndexSet(mymesh, topologicalIndex=0, size=mymesh.nodesGlobal, fromObject=airNodes)

In [ ]:
# density units [km*m^-3]
densityMap = { airParams['index']  : airParams['density'],
               stickyAirParams['index']  : stickyAirParams['density'],
               crustParams['index']  : crustParams['density'] + worldParams['thermalExp'] * (temperatureField-293.15) ,
               mantleParams['index']  : mantleParams['density'] + worldParams['thermalExp'] * (temperatureField-293.15) }

densityFn = fn.branching.map( fn_key = materialVariable, mapping = densityMap )

# heating functions for the materials [ W m^-3 / (rho * cp )] 
radiogenicMap = { airParams['index']  : 0.0, 
                  stickyAirParams['index']  : 0.0, 
                  crustParams['index']  : crustParams['radiogenic']/(densityFn*worldParams['cp']), 
                  mantleParams['index']  : mantleParams['radiogenic']/(densityFn*worldParams['cp']) }

radiogenicFn = fn.branching.map( fn_key = materialVariable, mapping = radiogenicMap )

# heat boundary condition
temperatureField.data[airSet.data] = 293.15
heatFlux = fn.misc.constant(worldParams['basalheatFlow'])

In [ ]:
tempDirichletBCs = uw.conditions.DirichletCondition( variable    = temperatureField, 
                                                     indexSetsPerDof = ( airSet ) )
tempNeumannBCs = uw.conditions.NeumannCondition( fn_flux=heatFlux, variable=temperatureField,
                                              indexSetsPerDof = (bottomWall) )

In [ ]:
heatSS = uw.systems.SteadyStateHeat(temperatureField, 
                                     fn_diffusivity=worldParams['diffusivity'],
                                     fn_heating=radiogenicFn,
                                     conditions=[tempDirichletBCs, tempNeumannBCs])

heatSSSolver = uw.systems.Solver(heatSS)

In [ ]:
heatSSSolver.solve(nonLinearIterate=True)

In [ ]:
tfig = glucifer.Figure()
tfig.append( glucifer.objects.Surface(mymesh, temperatureField))
tfig.show()

In [ ]:
rhofig = glucifer.Figure()
rhofig.append( glucifer.objects.Points(myswarm, densityFn, valueRange=[0.1,3400], logScale=True) )
rhofig.show()

In [ ]:
from math import fabs

class Node(object):
    def __init__(self, mesh, index):
        self.index = index
        self.x = mesh.data[index][0]
        self.y = mesh.data[index][1]

class Element(object):
    def __init__(self, mesh, index):
        self.index = index
        self.botLeftNode  = Node(mesh, mesh.data_elementNodes[index][0])
        self.botRightNode = Node(mesh, mesh.data_elementNodes[index][1])
        self.topLeftNode  = Node(mesh, mesh.data_elementNodes[index][2])
        self.topRightNode = Node(mesh, mesh.data_elementNodes[index][3])
        
        if index < mesh.elementsGlobal - mesh.elementRes[0]:
            
            self.above = index + mesh.elementRes[0]
        else:
            self.above = None
            
        if index > (mesh.elementRes[0] - 1):
            
            self.below = index - mesh.elementRes[0]
        else:
            self.below = None
        
        self.dX = fabs(self.botRightNode.x - self.botLeftNode.x)
        self.dY = fabs(self.topLeftNode.y - self.botRightNode.y)
        
        self.volume = self.dX * self.dY
        
        
def lithoPressure(mesh, lithoPress, Density, gravity):
    # Go through the mesh elements starting from the top right corner
    # !! Order matters !!
    for index in range(mesh.elementsGlobal - 1, -1, -1):
        elem = Element(mesh, index)
    
        pressure = 0.
        above = elem.above
    
        if above is not None: # Get Pressure from above elements
            pressure = lithoPress.data[above]
            elemAbove = Element(mesh, above)
            pressure += (gravity * elemAbove.dY / 4.0 * (Density.data[elemAbove.botLeftNode.index] +
                    Density.data[elemAbove.botRightNode.index]))
    
        pressure += (gravity * elem.dY / 4.0 * (Density.data[elem.topLeftNode.index] +
                Density.data[elem.topRightNode.index]))
        lithoPress.data[index] = pressure
        
    return lithoPress.data

DensityVar = uw.mesh.MeshVariable(mymesh, nodeDofCount=1)
projectorDensity = uw.utils.MeshVariable_Projection( DensityVar, densityFn, type=0 )
projectorDensity.solve()

pressureField.data[:] = lithoPressure(mymesh, pressureField, DensityVar, 9.81)

In [ ]:
pfig = glucifer.Figure()
pfig.append( glucifer.objects.Surface(mymesh, pressureField,logScale=False) )
pfig.show()

In [ ]:
def evalMelt( pressure, temperature ):
    T_s = np.polynomial.polynomial.polyval(pressure, [1063,-1.2e-7,1.2e-16])
    T_l = np.polynomial.polynomial.polyval(pressure, [1563.0,-1.2e-7,1.2e-16])
    T_ss = ( temperature - 0.5*(T_s+T_l) ) / (T_l-T_s)
    return np.where( (-0.5<T_ss) & (T_ss<0.5), 
                     0.5 + T_ss + ( T_ss*T_ss -0.25 )*( 0.4256 + 2.988 * T_ss ), 
                     0.0  )

In [ ]:
# test the melt function but rendering it on the mesh
meltField.data[:] = evalMelt (pressureField.evaluate(mymesh), temperatureField.evaluate(mymesh) )

mfig = glucifer.Figure()
mfig.append( glucifer.objects.Surface(mymesh, meltField) )
mfig.show()

In [ ]:
# make a geotherm along the MinI_VertexSet
wallNodes = mymesh.specialSets['MinI_VertexSet']

# get data: coords, temperature, pressure, mesh
ycoords = mymesh.data[wallNodes.data][:,1].reshape(-1,1)/1e3
temps = temperatureField.data[wallNodes.data]
pressure = pressureField.evaluate(wallNodes)
melt = meltField.data[wallNodes.data]

# use numpy to evaluate solidus and liquidus
T_s = np.polynomial.polynomial.polyval(pressure, [1063,-1.2e-7,1.2e-16])
T_l = np.polynomial.polynomial.polyval(pressure, [1563.0,-1.2e-7,1.2e-16])

%matplotlib inline
import matplotlib.pyplot as pyplot
import matplotlib.pylab as pylab
pylab.rcParams[ 'figure.figsize'] = 12, 6
pyplot.plot(temps,ycoords, 'o', color = 'black', label='geotherm') 
pyplot.plot(T_s,ycoords, 'o', color = 'blue', label='solidus') 
pyplot.plot(T_l,ycoords, 'o', color = 'red', label='liquidus') 

pyplot.ylabel('depth km')
pyplot.xlabel('Temperature K')
pyplot.legend()
pyplot.show()

In [ ]:
# example of what would be required for a restart of the above simulation

# velocityField = uw.mesh.MeshVariable(mymesh, nodeDofCount=3 )
# tDot = uw.mesh.MeshVariable(mymesh, nodeDofCount=1 )

# old_melt.data[:] = evalMelt(-1.0*pressureField.evaluate(myswarm), temperatureField.evaluate(myswarm))

# dF_dt = fn.misc.constant(0.0) # important for redefining later

# dynamicHeating = meltParams['lhf']/worldParams['cp']*dF_dt

# temperatureField.load('./path/to/saved_tempurature.h5')

# heatEq = uw.systems.AdvectionDiffusion(temperatureField, tDot, velocityField,  
#                                      fn_diffusivity=worldParams['diffusivity'],
#                                      fn_sourceTerm=radiogenicFn,# + temperatureField*dynamicHeating,
#                                      conditions=[tempDirichletBCs, tempNeumannBCs])

In [ ]:
# # setup the melting viscosity modifier as a function

# change = 1.0-(1.0-meltParams['viscosityChange'])/(0.15-0.3)*(meltField-0.15)

# meltViscosityFn = fn.branching.conditional( [ ( meltField < 0.15, 1.0 ),
#                                               ( meltField > 0.3 , 1e-3),
#                                               (  True, change )] )
